In [ ]:
!pip install faiss-cpu

In [96]:
import os
import faiss
import openai
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from langchain.chains import TransformChain
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain.schema import Document
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.output_parsers.structured import (
    ResponseSchema,
    StructuredOutputParser,
)
from langchain_core.prompts import (
    PromptTemplate,
)
from langchain_core.output_parsers import (
    StrOutputParser,
)

# 환경변수 불러오기
load_dotenv()

True

In [119]:
data = pd.read_csv('./중구_소공동_카페1.csv', encoding='utf-8')
data.iloc[3]

store_name                                                                웨스틴 조선 서울 조선델리
category                                                                            베이커리
new_open                                                                                
rating                                                                           별점 4.57
visited_review                                                                방문자 리뷰 754
directions_text                                                                         
store_id                                                                       558674555
address                                             서울 중구 소공로 106 지하 1층, 웨스틴 조선 서울 저층 로비
blog_review                                                                   블로그 리뷰 436
phone_num                                                                    02-317-0022
business_hours                         목\n07:00 - 21:00\n금\n07:00 - 21:00\n토\n07:00 -...
info                 

In [107]:
def split_into_domains(document):
    # Extract metadata and content
    metadata = document.metadata  # Use direct attribute access
    content = document.page_content  # Use direct attribute access

    # Parse content into structured data
    structured_data = {}
    for line in content.split("\n"):
        if ":" in line:
            key, value = line.split(":", 1)
            structured_data[key.strip()] = value.strip()

    # Create domain-specific metadata
    domains = {
        "store_info": {
            "store_name": structured_data.get("store_name"),
            "category": structured_data.get("category", "").split(","),
            "rating": structured_data.get("rating"),
            "address": structured_data.get("address"),
            "phone_num": structured_data.get("phone_num"),
            "sns": structured_data.get("sns"),
        },
        "business_info": {
            "business_hours": {
                k.strip(): v.strip()
                for k, v in zip(
                    structured_data.get("business_hours", "").split("\n")[::2],
                    structured_data.get("business_hours", "").split("\n")[1::2],
                )
            },
            "parking": structured_data.get("parking"),
            "convenience_facilities_and_services": structured_data.get(
                "convenience_facilities_and_services", ""
            ).split("\n"),
        },
        "review_info": {
            "visited_review": structured_data.get("visited_review"),
            "blog_review": structured_data.get("blog_review"),
            "review_like_this_part_output": structured_data.get("blog_review"),
        },
    }

    return domains

In [108]:
# CSVLoader 클래스를 사용하여 CSV 파일 로드
loader = CSVLoader(file_path='./중구_소공동_카페1.csv')  # CSV 파일 경로 지정

# CSV 파일에서 데이터 내용 로드
loaded_documents = loader.load()  # 로드된 데이터가 loaded_documents 변수에 저장

# Split documents into domain-specific data
domain_documents = [split_into_domains(document) for document in loaded_documents]

# Convert documents to a format suitable for FAISS
processed_documents = [
    {
        "page_content": "\n".join(f"{key}: {value}" for key, value in domain.items() if value),
        "metadata": {"domain": domain_name},
    }
    for document in domain_documents
    for domain_name, domain in document.items()
]

# Embedding and FAISS indexing
model_embedding = OpenAIEmbeddings(model='text-embedding-ada-002')
db = FAISS.from_documents(
    [Document(page_content=d["page_content"], metadata=d["metadata"]) for d in processed_documents],
    embedding=model_embedding,
)

# Save FAISS index
db.save_local("faiss_index")

In [109]:
retriever = db.as_retriever(
    searcy_type='mmr',
    search_kwargs={
        'fetch_k': 20,        # mmr에 전달할 문서 수
        'k': 6,               # 반환 문서 수
        'lambda_mult': 0.25,  # 다양성 조정 파라미터
        # 'filter': {'source': './data/keywords.txt'}        # 메타데이터 기반 필터링
    }
)
answer = retriever.invoke('서울에 있는 카페 하나 추천해줘')
print(answer[0].page_content)

store_name: 덕수궁 카페 돌담콩
category: ['크레페']
rating: 별점 4.71
address: 서울 중구 덕수궁길 9 현진빌딩 102호, 103호
phone_num: 0507-1419-7729
sns: https://blog.naver.com/doldamkong


In [122]:
# question =  '''
#             서울에 3박 4일동안 여행을 갈거야. 여행의 목적은 한국의 K-POP 관련 관광지를 방문하는 것이야. 
#             추천해줄 수 있어?
#             '''

question = '커피앤시가렛에 대한 정보를 알려줘'

template = '''
    너는 벡터 DB의 데이터만 사용해서 서울의 관광 일정을 추천해주는 봇이야.
    사용자의 질문에 따라서 여러가지 서울의 관광지, 식당, 숙소, 쇼핑몰 등을 추천해야해.
    처음에 사용자가 서울에 며칠동안 머무르는지 물어보고, 그에 따라서 관광 일정을 추천해주면 돼.
    그 다음 사용자가 서울에 오는 이유에 대해 파악하고 그에 맞는 여행 일정을 추천해주면 돼.
    대화하다가 추천한 여행 일정이 사용자의 마음에 들지 않아서 변경해달라고 하면 마음에 들지 않은 부분을
    캐치하고 그 부분들만 수정해서 다시 추천해주면 돼.
    일정 중간 중간 숙소나 식당, 쇼핑몰 등을 추천해주면 돼.

    일정에 대해 서로 대화하다가 어떤 장소가 궁금해서 물어본다면 특정 장소의 정보들을 정확하게 알려줘.
    (예:가게 정보, 메뉴, 별점, 위치, 연락처, 운영 시간, 리뷰 등)
    여행 일정에 대한 동선이 웬만하면 짧도록 추천해줘.
    언어는 사용자가 입력한 언어를 기준으로 알려줘. 
    화폐 기준도 사용자가 입력한 언어를 사용하는 국가의 화폐를 기준으로 적용해줘.

    질문: {question}

    내용: {reference}

    언어: {language}
'''

prompt = PromptTemplate(
    template=template,
    input_variables=['question', 'reference', 'language'],
)

model = ChatOpenAI(model='gpt-4o-mini')
parser = StrOutputParser()
chain = (
    {
        'reference': itemgetter('question') | retriever,
        'question': itemgetter('question'),
        'language': itemgetter('language'),
    }
    | prompt
    | model
    | parser
)

# chain을 통해 question
reference = chain.invoke(
    {
        'question': f'{question}',
        'language': '한국어',
    }
)

print(f"Answer: \n{reference}")

Answer: 
커피앤시가렛에 대한 정보는 다음과 같습니다:

- **가게 이름**: 커피앤시가렛
- **카테고리**: 카페
- **별점**: 4.43
- **주소**: 서울 중구 서소문로 116 유원빌딩 17층 1706호
- **전화번호**: 02-777-7557
- **웹사이트**: [커피앤시가렛 웹사이트](http://coffeeandcigarettes.kr/)

더 궁금한 점이 있다면 말씀해 주세요!
